
<h3 align="center">به نام خدا </h3>
<h2 align="center">تمرین سری چهارم </h2>


<p align="right">در این تمرین می خواهیم با کمک دانسته های خود از توزیع نرمال یک متغیره یک دسته بند طراحی کنیم</p>

<p align="right">برای پیاده سازی مدل ها از کتابخانه  استفاده کنید scikitlearn</p>

<p align="right">https://scikit-learn.org/stable/</p>

<p align="center">Taha Samavati - 98722134</p>

# خواندن داده ها

<p align="right">ابتدا داده ها را از کتابخانه می خوانیم و سپس داده ها را به دو دسته ی آموزش و آزمایش تقسیم بندی می کنیم</p>

In [1]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
y = iris.target

# store the feature matrix (X) and response vector (y) 
X = iris.data
y = iris.target
print("our dataset has " + str(X.shape[1]) + " features. for more information about data surf the web")
# splitting X and y into training and testing sets
#you can change the test size, fit model with more or less data and see results
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1) 


our dataset has 4 features. for more information about data surf the web


In [2]:
import numpy as np

### q1) print the number of train and test data and number of classes

In [3]:
print("the number of train data is : " + str(X_train.shape[0]))
print("the number of test data is : " + str(X_test.shape[0]))
print("there are " + str(len(set(y_train))) + " different classes in the dataset")

the number of train data is : 90
the number of test data is : 60
there are 3 different classes in the dataset


# آموزش یک مدل با فرض گوسی بودن توزیع داد ها

<p align="right">فرض کنید می دانیم که داده های ما از یک توزیع گوسی پیروی می کنند و همچنین ویژگی ها از هم مستقل هستند لطفا با کمک فرض های مساله و بدون کمک از مدل های آماده یک مدل برای دسته بندی کلاس ها آموزش دهید . دقت کنید که نباید از کدهای آماده استفاده کنید و فقط مجاز هستید برای فرمول های پایه مثل میانگین یا واریانس از توابع آماده استفاده نمایید</p>

In [4]:
# training the model on training set 

# write your code here :
# Naive Bayes model implementation
class Bayes():
    
    def __init__(self):
        self.priors = None
        self.distribution_params = None
    
    def gaussian(self,x,u,var):
        return (1/np.sqrt(2*np.pi*var)) * np.exp(-((x-u)**2)/(2*var))
    
    def _compute_priors(self,yt):

        classes = np.unique(yt)
        targets = np.asarray(yt)
        self.num_classes = (len(classes))
        # compute prior probabilities
        priors = np.zeros((len(classes),))
        for i in range(len(classes)):
            priors[i] = yt[yt==classes[i]].shape[0]/targets.shape[0]

        return priors

    def _estimate_pdf_dist(self, xt,yt):
        classes = np.unique(yt)
        targets = np.asarray(yt)
        distribution_params = np.zeros((len(classes),xt.shape[1],2))
        # compute class conditional pdfs
        for i in range(len(classes)):
            x = xt[yt==classes[i]]
            for j in range(x.shape[1]):
                distribution_params[i,j,0] = np.mean(x[:,j])
                distribution_params[i,j,1] = np.var(x[:,j])
        return distribution_params
    
    def train(self,xt,yt):
        self.priors = self._compute_priors(yt)
        self.distribution_params = self._estimate_pdf_dist(xt,yt)

    def predict(self,sample):
        # compute posterior probabilities
        posteriors = np.zeros((self.num_classes,))
        for i in range(self.num_classes):
            likelihoods = np.asarray([self.gaussian(x,u,var) for x,u,var in zip(sample,self.distribution_params[i,:,0],self.distribution_params[i,:,1]) ])
            posteriors[i] = self.priors[i]* np.prod(likelihoods)
        # return argument of maximum posterior
        return np.argmax(posteriors)


model = Bayes()
model.train(X_train,y_train)

<p align="right">اکنون با کمک داده های آزمایشی دقت مدل خود را بسنجید</p>

In [8]:
#write your code here :
predictions = []
for i in range(X_test.shape[0]):
    predictions.append(model.predict(X_test[i,:]))
    
from sklearn.metrics import precision_recall_fscore_support
prec,rcall,f1,_ = precision_recall_fscore_support(y_test, predictions,average = 'weighted')
print("Precision:"+str(round(prec*100,2))+"  Recall:"+str(round(rcall*100,2))+"  F1-score:"+str(round(f1*100,2)))

Precision:95.08  Recall:95.0  F1-score:95.0


# آموزش مدل بدون دانستن توزیع داده ها

<p align="right">داده ها را با کمک دسته بند اس وی ام و همچنین یک شبکه عصبی ساده دسته بندی کنید و دقت را با بخش قبل مقایسه نمایید</p>

In [10]:
from sklearn import svm
#train svm model
#write yor code here :
svm_model = svm.SVC()
svm_model.fit(X_train,y_train)
# predict
predictions = svm_model.predict(X_test)

# evaluate performance
prec,rcall,f1,_ = precision_recall_fscore_support(y_test, predictions,average = 'weighted')
print("Precision:"+str(round(prec*100,2))+"%  Recall:"+str(round(rcall*100,2))+"%  F1-score:"+str(round(f1*100,2))+"%")

Precision:98.41%  Recall:98.33%  F1-score:98.33%


In [11]:
from sklearn.neural_network import MLPClassifier
#use two hidden layers 
#train multi layer perceptron
#write yor code here :
neural_model = MLPClassifier(hidden_layer_sizes=(256,100 ), activation='logistic',max_iter=1000)
neural_model.fit(X_train,y_train)

# predict
predictions = neural_model.predict(X_test)

# evaluate
prec,rcall,f1,_ = precision_recall_fscore_support(y_test, predictions,average = 'weighted')
print("Precision:"+str(round(prec*100,2))+"%  Recall:"+str(round(rcall*100,2))+"%  F1-score:"+str(round(f1*100,2))+"%")

Precision:100.0%  Recall:100.0%  F1-score:100.0%


* The Precision of implemented Naive bayes on test-set is 95 % 
* The Precision of SVM classifier is better with 98.4%
* The Best precision is achieved with neural network model 100%

* **The precision of the implemented Gaussian Naive Bayes model is lower than other trained model because we naively assumed the data has gaussian distribution.**



---



# خواندن داده های جدید

In [12]:
wine = datasets.load_wine()
# store the feature matrix (X) and response vector (y) 
X = wine.data 
y = wine.target 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1) 

# آموزش یک مدل با فرض گوسی بودن توزیع داد ها

<p align="right">فرض کنید می دانیم که داده های ما از یک توزیع گوسی پیروی می کنند با کمک کتابخانه های آماده یک مدل برای دسته بندی کلاس ها آموزش دهید می  توانید برای این قسمت از کتابخانه های آماده نیز استفاده کنید و نیازی به پیاده سازی نیست</p>

In [13]:
from sklearn.naive_bayes import GaussianNB
# training the model on training set 

# write your code here :
gnb = GaussianNB()
gnb.fit(X_train,y_train)


GaussianNB(priors=None, var_smoothing=1e-09)

<p align="right">اکنون با کمک داده های آزمایشی دقت مدل خود را بسنجید</p>

In [14]:
#write your code here :
predictions = gnb.predict(X_test)

prec,rcall,f1,_ = precision_recall_fscore_support(y_test, predictions,average = 'weighted')
print("Precision:"+str(round(prec*100,2))+"%  Recall:"+str(round(rcall*100,2))+"%  F1-score:"+str(round(f1*100,2))+"%")

Precision:98.66%  Recall:98.61%  F1-score:98.61%


# آموزش مدل بدون دانستن توزیع داده ها

<p align="right">داده ها را با کمک دسته بند اس وی ام و همچنین یک شبکه عصبی ساده دسته بندی کنید و دقت را با بخش قبل مقایسه نمایید</p>

In [15]:
from sklearn import svm
#train svm model
#write yor code here :
svm_model = svm.SVC()
svm_model.fit(X_train,y_train)

# predict
predictions = svm_model.predict(X_test)

# evaluate
prec,rcall,f1,_ = precision_recall_fscore_support(y_test, predictions,average = 'weighted')
print("Precision:"+str(round(prec*100,2))+"%  Recall:"+str(round(rcall*100,2))+"%  F1-score:"+str(round(f1*100,2))+"%")

Precision:70.0%  Recall:70.83%  F1-score:69.76%


In [16]:
from sklearn.neural_network import MLPClassifier
#train multi layer perceptron
#write yor code here :
neural_model = MLPClassifier(hidden_layer_sizes=(256,100 ), activation='logistic',max_iter=1000)
neural_model.fit(X_train,y_train)

# predict
predictions = neural_model.predict(X_test)

# evaluate

prec,rcall,f1,_ = precision_recall_fscore_support(y_test, predictions,average = 'weighted')
print("Precision:"+str(round(prec*100,2))+"%  Recall:"+str(round(rcall*100,2))+"%  F1-score:"+str(round(f1*100,2))+"%")

Precision:94.74%  Recall:94.44%  F1-score:94.42%


* The Precision of GaussianNaiveBayes on test-set is 98.7 % 
* The Precision of SVM classifier is lower with 70.0%
* The Neural net precision is 94.74%
-----------------------------------------------
* The precision of the Gaussian Naive Bayes model is Better than other trained models
* **this shows that the data really has a gaussian like distribution**



---



#بخش امتیازی

<p align="right">آیا راهی وجود دارد که قبل از انتخاب بک دسته بند بتوان توزیع یک داده را حدس زد و دسته بندی را بر اساس این بینش از داده ها انجام داد؟</p>

* **The simplest way is to plot the data but this method can't help a lot.**
* **Another way is to use distribution tests such as "the Anderson-Darling test"**
* **These tests yield a p-value and if it's lower than some value we can reject the null hypothesis and conclude that our data doesn't have that special distribution, So we try other distributions to find one with high p-value.**